# 3. Converting data to OME-NGFF (practical)

***

**ELMI 2021 NGFF Workshop**, 25 June 2021

***


## Summary

* 3.1. Data from IDR
* 3.2. Choosing our data & downloading from S3
* 3.3. Converting your data to OME-NGFF
* 3.4. Publishing your data with S3
* 3.5. Extras (time-permitting)

***

## 3.1. Data from S3
We're going to start off by looking at some images you will likely have seen during the OMERO or IDR sessions.

**Our goal is to share these *without* using an OMERO.**

<table>
    <tr>
        <td>
            <img alt="idr0062 thumbnails" src="images/training-1.png" style="height:150px"/>
        </td>
        <td>
            <img alt="idr0062 thumbnails" src="images/training-2.png" style="height:150px"/>
        </td>
        <td>
            <img alt="idr0023 3D screenshot" src="images/training-3.png" style="height:150px"/>
        </td>
    </tr>
</table>
    
The left two images are from  the ilastik plugin guide presented by Petr: https://omero-guides.readthedocs.io/en/latest/ilastik/docs/ilastik_fiji.html

They are available in the "idr0062" project on the workshop server: https://workshop.openmicroscopy.org/webclient/?show=project-1952

The original dataset can be found in IDR study idr0062 by Blin _et al._: https://idr.openmicroscopy.org/webclient/?show=project-801

The image on the right is from idr0023 by Szymborska _et al_: http://idr.openmicroscopy.org/webclient/?show=project-52 and is **much** smaller.


***

## 3.2 Converting your data to OME-NGFF

The two basic commands are `bioformats2raw` and `raw2ometiff`. Together they provide a pipeline to scalably convert large images into OME-TIFF. The primary caveat is that they require **twice** the storage for the conversion.


### 3.2.1 Conversion tools

https://forum.image.sc/t/converting-whole-slide-images-to-ome-tiff-a-new-workflow/32110/4

<img src="images/blog-2019-12-converting-whole-slide-images.jpg" style="height:200px" />

<img src="images/conversion.png" style="height:400px" />

In [14]:
!bioformats2raw

Missing required parameters: <inputPath>, <outputPath>
Usage: <main class> [--debug] [--version] [--extra-readers[=<extraReaders>[,
                    <extraReaders>...]]]...
                    [--additional-scale-format-string-args=<additionalScaleForma
                    tStringArgsCsv>] [-c=<compressionType>]
                    [--compression-parameter=<compressionParameter>]
                    [--dimension-order=<dimensionOrder>]
                    [--file_type=<fileType>] [-h=<tileHeight>]
                    [--max_cached_tiles=<maxCachedTiles>]
                    [--max_workers=<maxWorkers>]
                    [--memo-directory=<memoDirectory>]
                    [--pyramid-name=<pyramidName>] [-r=<pyramidResolutions>]
                    [--scale-format-string=<scaleFormatString>]
                    [-w=<tileWidth>] <inputPath> <outputPath>
      <inputPath>    file to convert
      <outputPath>   path to the output pyramid directory
      --additional-scale-format-st

## Required OME-Zarr options
Two of the options are currently necessary to produce OME-Zarr data:
```
      --file_type=<fileType>
                     Tile file extension: n5, zarr (default: n5) [Can break
                       compatibility with raw2ometiff]
```
and
```
     --dimension-order=<dimensionOrder>
                     Override the input file dimension order in the output file
                       [Can break compatibility with raw2ometiff] (XYZCT,
                       XYZTC, XYCTZ, XYCZT, XYTCZ, XYTZC)
```

`--file_type` which produces Zarr output rather than N5 as the intermediate format. If we additionally pass the `--dimension-order` argument, then the intermediate result can be used directly by the ome-zarr library.

In [23]:
%%time
!bioformats2raw /tmp/trans_norm.tif /tmp/trans_norm_out --file_type=zarr --dimension-order=XYZCT

2021-01-29 13:09:59,730 [main] INFO  c.g.bioformats2raw.Converter - Output will be incompatible with raw2ometiff (pyramidName: data.zarr, scaleFormatString: %d/%d)
2021-01-29 13:10:00,503 [main] ERROR loci.formats.ImageReader -  *** One or more readers is misbehaving. See the debug output for more information. e.g.:
     loci.formats.in.APLReader@42f8285e -> java.lang.NullPointerException('null') ***
2021-01-29 13:10:00,561 [main] INFO  loci.formats.ImageReader - TiffDelegateReader initializing /tmp/trans_norm.tif
2021-01-29 13:10:00,564 [main] INFO  loci.formats.in.MinimalTiffReader - Reading IFDs
2021-01-29 13:10:00,614 [main] INFO  loci.formats.in.MinimalTiffReader - Populating metadata
2021-01-29 13:10:00,646 [main] INFO  loci.formats.in.TiffReader - Checking comment style
2021-01-29 13:10:00,649 [main] INFO  loci.formats.in.BaseTiffReader - Populating OME metadata
2021-01-29 13:10:00,699 [main] INFO  loci.formats.in.MinimalTiffReader - Reading IFDs
2021-01-29 13:10:00,728 [main] I

2021-01-29 13:10:01,634 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 13/571
2021-01-29 13:10:01,634 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1611922201634] time[0] tag[getTile]
2021-01-29 13:10:01,635 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 15, 0, 0] to /0/0
2021-01-29 13:10:01,635 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=15 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:01,637 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 16, 0, 0] to /0/0
2021-01-29 13:10:01,637 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 14/571
2021-01-29 13:10:01,637 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1611922201637] time[0] tag[getTile]
2021-01-29 13:10:01,639 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 16, 0, 0] to /0/0
2021-01-29 13:1

2021-01-29 13:10:01,685 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=31 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:01,686 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 32, 0, 0] to /0/0
2021-01-29 13:10:01,686 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 31/571
2021-01-29 13:10:01,686 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611922201686] time[0] tag[getTile]
2021-01-29 13:10:01,687 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 32, 0, 0] to /0/0
2021-01-29 13:10:01,687 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=32 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:01,689 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 33, 0, 0] to /0/0
2021-01-29 13:10:01,689 [pool-1-thread-2] INFO  c.g.bioformats2raw.Conv

2021-01-29 13:10:01,737 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 50, 0, 0] to /0/0
2021-01-29 13:10:01,737 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=50 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:01,738 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 51, 0, 0] to /0/0
2021-01-29 13:10:01,738 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 50/571
2021-01-29 13:10:01,738 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611922201738] time[0] tag[getTile]
2021-01-29 13:10:01,739 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 51, 0, 0] to /0/0
2021-01-29 13:10:01,740 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=51 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:01,741 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter 

2021-01-29 13:10:01,788 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 70, 0, 0] to /0/0
2021-01-29 13:10:01,788 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=70 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:01,789 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 72, 0, 0] to /0/0
2021-01-29 13:10:01,789 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - tile read complete 70/571
2021-01-29 13:10:01,789 [pool-1-thread-3] INFO  org.perf4j.TimingLogger - start[1611922201789] time[0] tag[getTile]
2021-01-29 13:10:01,790 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 72, 0, 0] to /0/0
2021-01-29 13:10:01,790 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=72 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:01,791 [pool-1-thread-3] INFO  c.g.bioformats2raw.Converter 

...

2021-01-29 13:10:02,404 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 418, 0, 0] to /0/0
2021-01-29 13:10:02,404 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=418 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:02,405 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 419, 0, 0] to /0/0
2021-01-29 13:10:02,405 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - tile read complete 418/571
2021-01-29 13:10:02,405 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1611922202405] time[0] tag[getTile]
2021-01-29 13:10:02,405 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 419, 0, 0] to /0/0
2021-01-29 13:10:02,405 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=419 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:02,406 [pool-1-thread-4] INFO  c.g.bioformats2raw.Conv

2021-01-29 13:10:02,455 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 461, 0, 0] to /0/0
2021-01-29 13:10:02,455 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 460/571
2021-01-29 13:10:02,455 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611922202455] time[0] tag[getTile]
2021-01-29 13:10:02,456 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 461, 0, 0] to /0/0
2021-01-29 13:10:02,456 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=461 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:02,456 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 462, 0, 0] to /0/0
2021-01-29 13:10:02,457 [pool-1-thread-2] INFO  c.g.bioformats2raw.Converter - tile read complete 461/571
2021-01-29 13:10:02,457 [pool-1-thread-2] INFO  org.perf4j.TimingLogger - start[1611922202456] time[0] tag[getTile]
202

2021-01-29 13:10:02,506 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 500, 0, 0] to /0/0
2021-01-29 13:10:02,506 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 499/571
2021-01-29 13:10:02,506 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1611922202506] time[0] tag[getTile]
2021-01-29 13:10:02,507 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 500, 0, 0] to /0/0
2021-01-29 13:10:02,507 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=500 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:02,508 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 501, 0, 0] to /0/0
2021-01-29 13:10:02,508 [pool-1-thread-1] INFO  c.g.bioformats2raw.Converter - tile read complete 500/571
2021-01-29 13:10:02,508 [pool-1-thread-1] INFO  org.perf4j.TimingLogger - start[1611922202508] time[0] tag[getTile]
202

2021-01-29 13:10:02,557 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 533, 0, 0] to /0/0
2021-01-29 13:10:02,557 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=533 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:02,558 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 534, 0, 0] to /0/0
2021-01-29 13:10:02,558 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - tile read complete 532/571
2021-01-29 13:10:02,558 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1611922202558] time[0] tag[getTile]
2021-01-29 13:10:02,559 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 534, 0, 0] to /0/0
2021-01-29 13:10:02,559 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=534 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:02,560 [pool-1-thread-4] INFO  c.g.bioformats2raw.Conv

2021-01-29 13:10:02,609 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 566, 0, 0] to /0/0
2021-01-29 13:10:02,610 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - tile read complete 564/571
2021-01-29 13:10:02,610 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1611922202609] time[0] tag[getTile]
2021-01-29 13:10:02,610 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - successfully wrote at [0, 0, 566, 0, 0] to /0/0
2021-01-29 13:10:02,610 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - Successfully processed tile; resolution=0 plane=566 xx=0 yy=0 width=30 height=30
2021-01-29 13:10:02,611 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - requesting tile to write at [0, 0, 567, 0, 0] to /0/0
2021-01-29 13:10:02,611 [pool-1-thread-4] INFO  c.g.bioformats2raw.Converter - tile read complete 565/571
2021-01-29 13:10:02,611 [pool-1-thread-4] INFO  org.perf4j.TimingLogger - start[1611922202611] time[0] tag[getTile]
202

In [24]:
!ls /tmp/trans_norm_out

METADATA.ome.xml data.zarr


In [25]:
!find /tmp/trans_norm_out/data.zarr -name ".z*"

/tmp/trans_norm_out/data.zarr/.zattrs
/tmp/trans_norm_out/data.zarr/.zgroup
/tmp/trans_norm_out/data.zarr/0/.zattrs
/tmp/trans_norm_out/data.zarr/0/.zgroup
/tmp/trans_norm_out/data.zarr/0/0/.zarray


In [30]:
!ome_zarr info /tmp/trans_norm_out/data.zarr/0

/private/tmp/trans_norm_out/data.zarr/0 [zgroup]
 - metadata
   - Multiscales
 - data
   - (1, 1, 571, 30, 30)


***

## 4. Publishing your data with S3

You can then move the generated output to S3. Note: one of the most frequent mistakes here is the slash (`/`) at the end of the commands.x

In [31]:
YOURNAME = input()

josh


In [32]:
!time mc cp --recursive /tmp/trans_norm_out/data.zarr/0/ gbi/bioim/$YOURNAME/my_trans_norm.ome.zarr/

....0.99.0.0:  772.90 KiB / 772.90 KiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  33.42 KiB/s 23s
real	0m23.704s
user	0m0.615s
sys	0m0.523s


In [33]:
!mc cat gbi/bioim/$YOURNAME/my_trans_norm.ome.zarr/.zattrs

{"multiscales":[{"datasets":[{"path":"0"}],"version":"0.1"}]}

In [35]:
!ome_zarr info /tmp/trans_norm_out/data.zarr/0

/private/tmp/trans_norm_out/data.zarr/0 [zgroup]
 - metadata
   - Multiscales
 - data
   - (1, 1, 571, 30, 30)


In [37]:
!ome_zarr info https://s3.embl.de/bioim/josh/my_trans_norm.ome.zarr/

https://s3.embl.de/bioim/josh/my_trans_norm.ome.zarr/ [zgroup]
 - metadata
   - Multiscales
 - data
   - (1, 1, 571, 30, 30)


This viewer is [vizarr](https://github.com/hms-dbmi/vizarr) from the [Gehlenborg lab](http://gehlenborglab.org/) at Harvard Medical School. It can be accessed at https://hms-dbmi.github.io/vizarr for example to access data from the IDR: [link](http://hms-dbmi.github.io/vizarr/v0.1?source=https%3A%2F%2Fs3.embassy.ebi.ac.uk%2Fidr%2Fzarr%2Fv0.1%2F6001240.zarr). But here, we've copied the static website to our bucket (https://s3.embl.de/bioim) to have a standalone copy.

## 5. Extras (time-permitting)

## 5.1 A larger example (idr0062)

In [23]:
from IPython.display import IFrame
IFrame("https://s3.embl.de/bioim/index.html?source=idr0062-zarrs/6001240.zarr/", width=700, height=350)

In [1]:
from IPython.display import IFrame
IFrame("https://hms-dbmi.github.io/vizarr/v0.2/?source=https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/9822151.zarr", width=700, height=350)

# 5.2 Renaming

Another important distinction to filesystems is that though it looks like hello is in a directory, you should really think of the entire string after the bucket just as a "key".

In [24]:
!mc mv --recursive gbi/bioim/$YOURNAME/my_trans_norm.ome.zarr/ gbi/bioim/$YOURNAME/public_trans_norm.ome.zarr

....0.99.0.0:  772.90 KiB / 772.90 KiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  63.15 KiB/s 12s

## 5.3 omero-cli-zarr

Loading the image from: https://workshop.openmicroscopy.org/webclient/img_detail/55500/?dataset=6210

In [25]:
# Another block to get your workshop password from day 1
import getpass
workshop_pass = getpass.getpass()

········


In [26]:
!omero login trainer-1@wss://workshop.openmicroscopy.org/omero-ws -w $workshop_pass

Previous session expired for trainer-1 on wss://workshop.openmicroscopy.org/omero-ws:4064
Created session for trainer-1@wss://workshop.openmicroscopy.org/omero-ws:443. Idle timeout: 10 min. Current group: Lab1


In [27]:
!time omero zarr export Image:55500

Using session for trainer-1@wss://workshop.openmicroscopy.org/omero-ws:443. Idle timeout: 10 min. Current group: Lab1
Exporting to 55500.zarr
Finished.

real	2m12.037s
user	0m5.787s
sys	0m1.844s


In [28]:
!find 55500.zarr -name ".z*"

55500.zarr/.zattrs
55500.zarr/.zgroup
55500.zarr/0/.zarray
55500.zarr/1/.zarray
55500.zarr/2/.zarray


## 5.4 Other resources

<table>
    <tr>
        <td>
            <a href="https://downloads.openmicroscopy.org/presentations/2020/Dundee/Workshops/NGFF/zarr_diagram/">
<img src="images/resources-1.png" alt="Screenshot of the Zarr diagram from OME2020" style="height:200px"/>
            </a>
        </td>
        <td>
<a href="https://downloads.openmicroscopy.org/presentations/2020/Dundee/Workshops/NGFF/zarr_diagram/">Diagram for how data moves</a>
        </td>
    </tr>
    <tr>
        <td>
      <a href="https://blog.openmicroscopy.org/file-formats/community/2020/11/04/zarr-data/">      
<img src="images/resources-2.png" alt="Screenshot of the Zarr diagram from OME2020" style="height:200px"/>
            </a>
        </td>
        <td>
<a href="https://blog.openmicroscopy.org/file-formats/community/2020/11/04/zarr-data/">Blog post for an easy way to publish OME-Zarr files</a>
        </td>
    </tr>
</table>    

### License
Copyright (C) 2021 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.